In [3]:
import os
from urllib.parse import quote

from bs4 import BeautifulSoup
import requests

BASE_URL = 'http://www.imsdb.com'
corpus_path = 'C:/Users/Rik/Documents/corpus/'
scripts_path = 'imsdb_parsed'

In [4]:
def clean_script(text):
    text = text.replace('Back to IMSDb', '')
    text = text.replace('''<b><!--
</b>if (window!= top)
top.location.href=location.href
<b>// -->
</b>
''', '')
    text = text.replace('''          Scanned by http://freemoviescripts.com
          Formatting by http://simplyscripts.home.att.net
''', '')
    return text.replace(r'\r', '')

In [5]:
def get_script(relative_link):
    tail = relative_link.split('/')[-1]
    print('fetching %s' % tail)
    script_front_url = BASE_URL + quote(relative_link)
    front_page_response = requests.get(script_front_url)
    front_soup = BeautifulSoup(front_page_response.text, "html.parser")

    try:
        script_link = front_soup.find_all('p', align="center")[0].a['href']
    except IndexError:
        print('%s has no script :(' % tail)
        return None, None

    if script_link.endswith('.html'):
        title = script_link.split('/')[-1].split(' Script')[0]
        script_url = BASE_URL + script_link
        script_soup = BeautifulSoup(requests.get(script_url).text, "html.parser")
        script_text = script_soup.find_all('td', {'class': "scrtext"})[0].get_text()
        script_text = clean_script(script_text)
        return title, script_text
    else:
        print('%s is a pdf :(' % tail)
        return None, None

In [7]:
response = requests.get('http://www.imsdb.com/all%20scripts/')
html = response.text

soup = BeautifulSoup(html, "html.parser")
paragraphs = soup.find_all('p')

for p in paragraphs:
    relative_link = p.a['href']
    title, script = get_script(relative_link)
    if not script:
        continue

    with open(os.path.join(corpus_path, scripts_path, title.strip('.html') + '.txt'), 'w') as outfile:
        outfile.write(script)

fetching 10 Things I Hate About You Script.html
fetching 12 Script.html
fetching 12 and Holding Script.html
fetching 12 Monkeys Script.html
fetching 12 Years a Slave Script.html
fetching 127 Hours Script.html
fetching 1492: Conquest of Paradise Script.html
fetching 15 Minutes Script.html
fetching 17 Again Script.html
fetching 187 Script.html
fetching 2001: A Space Odyssey Script.html
fetching 2012 Script.html
fetching 25th Hour Script.html
fetching 28 Days Later Script.html
fetching 30 Minutes or Less Script.html
fetching 42 Script.html


UnicodeEncodeError: 'charmap' codec can't encode characters in position 5078-5079: character maps to <undefined>